In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
import sys
sys.path.append('../')

In [3]:
from bricks_dataset.brick_envs.generative_env import GenerativeEnv


[robosuite WARNING] No private macro file found! (__init__.py:7)
[robosuite WARNING] It is recommended to use a private macro file (__init__.py:8)
[robosuite WARNING] To setup, run: python /home/user/anaconda3/envs/robosuite/lib/python3.9/site-packages/robosuite/scripts/setup_macros.py (__init__.py:9)


In [4]:
import copy

In [5]:
def generate_brick_shapes(max_size, num_blocks, memory):

    # print(max_size, num_blocks)

    if memory[max_size, num_blocks] is not None:
        return memory[max_size, num_blocks]
    if num_blocks == 0:
        shape = {}
        for i in range(1, max_size+1):
            shape[i] = 0
        memory[max_size, num_blocks] = [shape]
    elif max_size == 1:
        # only one option left
        memory[max_size, num_blocks] = [{1: num_blocks}]
    else:
        all_block_shapes = []
        for i in range(num_blocks + 1):
            # NOTE we need deepcopy here because otherwise we will modify the same dictionary
            block_shapes = copy.deepcopy(generate_brick_shapes(max_size - 1, num_blocks - i, memory))
            # TODO not so optimal => makes the algorithm super slow
            for other_block_shape in block_shapes:
                other_block_shape[max_size] = i
            all_block_shapes += block_shapes
        memory[max_size, num_blocks] = all_block_shapes

    return memory[max_size, num_blocks]

In [6]:
import math

In [85]:
block_sizes = ["1", "2", "3", "4", "2x2", "2x3", "2x4", "3x3", "3x4", "4x4"]

In [7]:
num_blocks = 10
max_size = 10
memory = np.empty((max_size + 1, num_blocks + 1), dtype="object")
brick_configs = generate_brick_shapes(max_size, num_blocks, memory)
brick_config_hashes = []
for brick_config in brick_configs:
    brick_config_hash = ""
    for brick_size in brick_config.keys():
        brick_config_hash += f"{brick_size}:{brick_config[brick_size]}|"
    brick_config_hashes.append(brick_config_hash)
# check there are no duplicates
assert len(brick_config_hashes) == len(set(brick_config_hashes))
# check that the number of bricks adds up to num_blocks
assert sum([sum(brick_config.values()) != num_blocks for brick_config in brick_configs]) == 0
# check that keys are present in all configs
assert sum([sum([k not in brick_config.keys() for k in range(1, max_size+1)]) for brick_config in brick_configs]) == 0
# check that the number of combinations is as expected
assert len(brick_configs) == math.comb(max_size + num_blocks - 1, num_blocks)

In [8]:
len(brick_configs)

92378

In [9]:
brick_configs

[{1: 10, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 9, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 8, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 7, 2: 3, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 6, 2: 4, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 5, 2: 5, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 4, 2: 6, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 3, 2: 7, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 2, 2: 8, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 1, 2: 9, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 0, 2: 10, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 9, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 8, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 7, 2: 2, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 6, 2: 3, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
 {1: 5, 2: 4, 3: 1, 4: 0, 5: 0, 6: 0, 

In [52]:
len(brick_config_hashes)

126

In [53]:
len(set(brick_config_hashes))

6

In [44]:
brick_configs

[{1: 5, 2: 0, 3: 0, 4: 0, 5: 0},
 {1: 4, 2: 0, 3: 0, 4: 0, 5: 1},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 4, 2: 0, 3: 0, 4: 0, 5: 1},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 4, 2: 0, 3: 0, 4: 0, 5: 1},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 2, 2:

[{1: 5, 2: 0, 3: 0, 4: 0, 5: 0},
 {1: 4, 2: 0, 3: 0, 4: 0, 5: 1},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 4, 2: 0, 3: 0, 4: 0, 5: 1},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 4, 2: 0, 3: 0, 4: 0, 5: 1},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 3, 2: 0, 3: 0, 4: 0, 5: 2},
 {1: 2, 2: 0, 3: 0, 4: 0, 5: 3},
 {1: 1, 2: 0, 3: 0, 4: 0, 5: 4},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 5},
 {1: 2, 2:

In [18]:
brick_shapes[max_size, num_blocks]

TypeError: list indices must be integers or slices, not tuple

In [13]:
memory[:, 0]

array([None, None, None, None, None, None], dtype=object)

In [8]:
brick_shapes

-1.0

In [7]:
brick_shapes[max_size, num_blocks]

IndexError: invalid index to scalar variable.

In [4]:
env = GenerativeEnv(num_blocks=7, total_num_pins=(1, 27, 1), base_shape=(4, 4))

TypeError: __init__() got an unexpected keyword argument 'env_configuration'

In [482]:
import cProfile

In [483]:
with cProfile.Profile() as pr:
    env.reset()

In [484]:
pr.print_stats()


         62532 function calls (51420 primitive calls) in 0.794 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
       14    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(concatenate)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(copy)
        7    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(copyto)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(dot)
      148    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(norm)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(reshape)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(vstack)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
       33   

In [481]:
%timeit env._reset_internal()

1.71 ms ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [434]:
def env_factory():
    env = None
    while True:
        try:
            env = GenerativeEnv(num_blocks=7, total_num_pins=(1, 27, 1), base_shape=(4, 4))
            break
        except:
            continue
    return env

In [10]:
%timeit env_factory()

1.43 s ± 9.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
torch.cuda.is_available()

True

In [442]:
def generate_xy(x_range, y_range, radii, max_seeds=200, verbose=False):
    num_objects = len(radii)
    radii = torch.Tensor(radii)

    # tensor for border
    b_x_min = radii + x_range[0]
    b_x_max = x_range[1] - radii
    b_y_min = radii + y_range[0]
    b_y_max = y_range[1] - radii

    # tensor for radius
    min_dist = torch.Tensor(radii)[:, None].repeat(1, num_objects) - torch.diag(radii)

    def check_distances(p_x, p_y):
        dx = p_x[:, None] - p_x[None, :]
        dy = p_y[:, None] - p_y[None, :]
        distances = min_dist**2 - (dx**2 + dy**2) > 0
        if verbose:
            print(distances)
        return torch.sum(distances) == 0
    
    def check_boundaries(p_x, p_y):
        borders = (p_x < b_x_min) + (p_x > b_x_max) + (p_y < b_y_min) + (p_y > b_y_max)
        if verbose:
            print("border")
            print(borders)
        return torch.sum(borders) == 0
    
    def compute_loss(p_x, p_y):
        dx = p_x[:, None] - p_x[None, :]
        dy = p_y[:, None] - p_y[None, :]
        border_loss = torch.sum(
            (b_x_min - torch.clamp(p_x, max=b_x_min))**2 + (b_x_max - torch.clamp(p_x, min=b_x_max))**2 + 
            (b_y_min - torch.clamp(p_y, max=b_y_min))**2 + (b_y_max - torch.clamp(p_y, min=b_y_max))**2
        )
        overlap_loss = torch.sum(
            min_dist**2 - (torch.clamp(dx**2, max=min_dist**2) + torch.clamp(dy**2, max=min_dist**2))
        )
        return 1e6 * border_loss + overlap_loss

    # initialize positions
    def init_params_and_optimizer():
        p_x = torch.rand(num_objects) * (b_x_max - b_x_min) + b_x_min
        p_y = torch.rand(num_objects) * (b_y_max - b_y_min) + b_y_min
        p_x.requires_grad = True
        p_y.requires_grad = True
        optimizer = torch.optim.Adam([p_x, p_y], lr=0.1)
        return p_x, p_y, optimizer
    
    # optimize
    seed_iter = 0
    while seed_iter < max_seeds:
        p_x, p_y, optimizer = init_params_and_optimizer()
        for i in range(20):
            optimizer.zero_grad()
            loss = compute_loss(p_x, p_y)
            loss.backward()
            optimizer.step()
            if verbose:
                print(i, loss.item())
            if i % 10 == 0 and check_distances(p_x, p_y) and check_boundaries(p_x, p_y):
                return p_x.detach().numpy(), p_y.detach().numpy(), i, seed_iter
        seed_iter += 1
            
    return None, None, None, None


In [193]:
a = torch.arange(3)
a[:, None] - a[None, :]

tensor([[ 0, -1, -2],
        [ 1,  0, -1],
        [ 2,  1,  0]])

In [409]:
p_x, p_y, i, seed_iter = generate_xy(x_range=(0, 1), y_range=(0, 1), radii=[0.1, 0.1, 0.2, 0.2, 0.1, 0.1], verbose=False)
print(i, seed_iter, p_x, p_y)

0 51 [0.2678014  0.15151897 0.28049138 0.6669462  0.23878789 0.7294684 ] [0.27693623 0.1496677  0.65294933 0.6508886  0.88753664 0.411539  ]


In [179]:
torch.ones((3, 1)) * (torch.arange(3) * 2)

tensor([[0., 2., 4.],
        [0., 2., 4.],
        [0., 2., 4.]])

In [169]:
p_x.shape

(6, 6)

In [167]:
p_x, p_y

(array([[0.28207186, 0.28207186, 0.33655393, 0.28207186, 0.28207186,
         0.28207186],
        [0.7140587 , 0.7140587 , 0.7276387 , 0.7140587 , 0.410018  ,
         0.7140587 ],
        [0.6657224 , 0.4108284 , 0.62429184, 0.66710573, 0.41082498,
         0.7222516 ],
        [0.88906306, 0.88906306, 0.24482343, 0.88906306, 0.88906306,
         0.3527751 ],
        [0.684146  , 0.7757483 , 0.71409714, 0.684146  , 0.684146  ,
         0.684146  ],
        [0.8170085 , 0.8170085 , 0.28189197, 0.21648651, 0.8170085 ,
         0.8170085 ]], dtype=float32),
 array([[0.49218622, 0.49218622, 0.49413973, 0.49218622, 0.49218622,
         0.49218622],
        [0.3530238 , 0.3530238 , 0.3897679 , 0.3530238 , 0.3530238 ,
         0.3530238 ],
        [0.8672121 , 0.8672121 , 0.7754091 , 0.8672121 , 0.41134953,
         0.8672121 ],
        [0.6251346 , 0.6251346 , 0.593851  , 0.6251346 , 0.6251346 ,
         0.6251346 ],
        [0.82539165, 0.82539165, 0.7873987 , 0.82539165, 0.82539165,
    

In [166]:
%timeit generate_xy(x_range=(0, 1), y_range=(0, 1), radii=[0.1, 0.1, 0.2, 0.1, 0.1, 0.1])

48.1 ms ± 83.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [139]:
a = torch.Tensor([1, 2, 3])
a.shape

torch.Size([3])

In [145]:
a[:, None].repeat(1, 3)

tensor([[1., 1., 1.],
        [2., 2., 2.],
        [3., 3., 3.]])

In [140]:
a.requires_grad

False

In [126]:
p_x = torch.rand((6, 1))
p_x.requires_grad = True

In [127]:
p_y = torch.rand((6, 1))
p_y.requires_grad = True

In [117]:
min_x = torch.ones((6, 6)) * 0.2 - torch.diag(torch.ones(6) * 0.2)
min_y = torch.ones((6, 6)) * 0.2 - torch.diag(torch.ones(6) * 0.2)

In [133]:
def check_distances(p_x, p_y):
    dx = p_x - p_x.T
    dy = p_y - p_y.T
    return min_x**2 - (dx**2 + dy**2) > 0
check_distances(p_x, p_y)

tensor([[False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False]])

In [129]:
def compute_loss(p_x, p_y):
    dx = p_x - p_x.T
    dy = p_y - p_y.T
    border_loss = torch.sum(
        torch.clamp(p_x, max=0)**2 + (1 - torch.clamp(p_x, min=1))**2 + 
        torch.clamp(p_y, max=0)**2 + (1 - torch.clamp(p_y, min=1))**2
    )
    overlap_loss = torch.sum(
        min_x**2 - (torch.clamp(dx**2, max=min_x**2) + torch.clamp(dy**2, max=min_y**2))
    )
    return 1e6 * border_loss + overlap_loss

In [130]:
optimizer = torch.optim.Adam([p_x, p_y], lr=0.01, betas=(0.9, 0.999), eps=1e-08)

In [135]:
def train_loop():
    for i in range(100):
        loss = compute_loss(p_x, p_y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # print(loss.item())

In [136]:
%timeit train_loop()

49.3 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [132]:
train_loop()

-0.2564101219177246
-0.3395269513130188
-0.43419474363327026
-0.5121573209762573
-0.5658945441246033
-0.6451884508132935
-0.687014102935791
-0.7113868594169617
-0.730811595916748
-0.7387544512748718
-0.7489892840385437
-0.7616831064224243
-0.7757910490036011
-0.7872983813285828
-0.8017213344573975
-0.8193690180778503
-0.8354816436767578
-0.8557793498039246
-0.8669878840446472
-0.8821569681167603
-0.894079327583313
-0.9091534614562988
-0.9277666807174683
-0.9473664164543152
-0.9679537415504456
-0.9663479328155518
0.062020182609558105
-0.9732017517089844
-0.9698249697685242
-0.967251718044281
-0.9693603515625
-0.9763658046722412
-0.9855645895004272
-0.9935724139213562
-0.9896829128265381
-0.9870574474334717
-0.9877191781997681
-0.9914126992225647
-0.996990442276001
-1.0035724639892578
-1.0094865560531616
-1.011838674545288
-1.0167529582977295
-1.0246856212615967
-1.034891963005066
-1.0370968580245972
-1.03690505027771
-1.038012146949768
-1.0403249263763428
-1.04380464553833
-1.0474240779

In [98]:
p_x

tensor([[0.2720],
        [0.0522],
        [0.4905],
        [0.7190],
        [0.9458],
        [0.0505]], requires_grad=True)

In [125]:
check_distances(p_x, p_y)

tensor([[False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False]])

In [85]:
dy

tensor([[ 0.0000e+00, -3.4449e-01,  0.0000e+00, -3.4449e-01, -5.5954e-01,
         -2.9802e-08],
        [ 3.4449e-01,  0.0000e+00,  3.4449e-01, -5.9605e-08, -2.1505e-01,
          3.4449e-01],
        [ 0.0000e+00, -3.4449e-01,  0.0000e+00, -3.4449e-01, -5.5954e-01,
         -2.9802e-08],
        [ 3.4449e-01,  5.9605e-08,  3.4449e-01,  0.0000e+00, -2.1505e-01,
          3.4449e-01],
        [ 5.5954e-01,  2.1505e-01,  5.5954e-01,  2.1505e-01,  0.0000e+00,
          5.5954e-01],
        [ 2.9802e-08, -3.4449e-01,  2.9802e-08, -3.4449e-01, -5.5954e-01,
          0.0000e+00]], grad_fn=<SubBackward0>)

In [79]:
p_y

tensor([[0.4178],
        [0.7629],
        [0.4184],
        [0.7628],
        [0.9779],
        [0.4188]], requires_grad=True)

In [81]:
train_loop()

1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.600000023841858
1.60000002

In [137]:
from robosuite.utils.placement_samplers import ObjectPositionSampler


In [447]:
radii = []
for b in env.bricks:
    radii.append(b.horizontal_radius)
radii

[0.07071067811865477,
 0.03952847075210475,
 0.027950849718747374,
 0.01767766952966369,
 0.06373774391990981,
 0.10077822185373188,
 0.03952847075210475,
 0.03952847075210475]

In [448]:
table_full_size = env.table_config.full_size

x_range = [0, table_full_size[0] / 2]
y_range = [-table_full_size[1] / 2, table_full_size[1] / 2]

print(x_range, y_range)
# self.placement_initializer = UniformRandomSampler(
#     name="BricksSampler",
#     mujoco_objects=self.bricks,
#     x_range=[0, table_full_size[0] / 2],
#     y_range=[-table_full_size[1] / 2, table_full_size[1] / 2],
#     rotation=None,
#     ensure_object_boundary_in_range=True,
#     ensure_valid_placement=True,
#     reference_pos=self.table_config.offset,
#     z_offset=0.01,
# )

[0, 0.4] [-0.4, 0.4]


In [455]:
p_x, p_y, i, seed_iter = generate_xy(x_range=x_range, y_range=y_range, radii=radii, verbose=False)
print(i, seed_iter, p_x, p_y)

10 154 [0.14796612 0.23803502 0.0814719  0.1927555  0.07916537 0.20099576
 0.34996507 0.07566661] [ 0.32249787  0.19797121  0.16022451 -0.08223852 -0.1717758   0.05491526
  0.15961677  0.0033053 ]


In [429]:
%timeit generate_xy(x_range=x_range, y_range=y_range, radii=radii, verbose=False)

302 ms ± 77.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [463]:
def sample_old():
    while True:
        try:
            object_placements = env.placement_initializer.sample()
            break
        except:
            continue
    return object_placements

In [477]:
sample_old()

{'brick_0': ((0.31989694823665027, 0.19074995638830866, 0.8225),
  array([0.99343692, 0.        , 0.        , 0.11438132]),
  <bricks_dataset.brick_objects.brick_obj.BrickObj at 0x7f05c3310d00>),
 'brick_1': ((0.12273583795073055, 0.24734252811559432, 0.8225),
  array([0.99569916, 0.        , 0.        , 0.09264546]),
  <bricks_dataset.brick_objects.brick_obj.BrickObj at 0x7f05c3310d30>),
 'brick_2': ((0.19844000103366924, 0.2479931128657138, 0.8225),
  array([0.3785078, 0.       , 0.       , 0.9255981]),
  <bricks_dataset.brick_objects.brick_obj.BrickObj at 0x7f05bc97edf0>),
 'brick_3': ((0.09480223955382142, -0.2830503946012521, 0.8225),
  array([0.39973776, 0.        , 0.        , 0.91662954]),
  <bricks_dataset.brick_objects.brick_obj.BrickObj at 0x7f05bc97e1c0>),
 'brick_4': ((0.1239723459078774, 0.14224329127959273, 0.8225),
  array([0.36085777, 0.        , 0.        , 0.93262086]),
  <bricks_dataset.brick_objects.brick_obj.BrickObj at 0x7f05bc97e580>),
 'brick_5': ((0.2462695248

In [462]:
%timeit sample_old()

764 µs ± 90.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
class UniformRandomSampler(ObjectPositionSampler):
    """
    Places all objects within the table uniformly random.

    Args:
        name (str): Name of this sampler.

        mujoco_objects (None or MujocoObject or list of MujocoObject): single model or list of MJCF object models

        x_range (2-array of float): Specify the (min, max) relative x_range used to uniformly place objects

        y_range (2-array of float): Specify the (min, max) relative y_range used to uniformly place objects

        rotation (None or float or Iterable):
            :`None`: Add uniform random random rotation
            :`Iterable (a,b)`: Uniformly randomize rotation angle between a and b (in radians)
            :`value`: Add fixed angle rotation

        rotation_axis (str): Can be 'x', 'y', or 'z'. Axis about which to apply the requested rotation

        ensure_object_boundary_in_range (bool):
            :`True`: The center of object is at position:
                 [uniform(min x_range + radius, max x_range - radius)], [uniform(min x_range + radius, max x_range - radius)]
            :`False`:
                [uniform(min x_range, max x_range)], [uniform(min x_range, max x_range)]

        ensure_valid_placement (bool): If True, will check for correct (valid) object placements

        reference_pos (3-array): global (x,y,z) position relative to which sampling will occur

        z_offset (float): Add a small z-offset to placements. This is useful for fixed objects
            that do not move (i.e. no free joint) to place them above the table.
    """

    def __init__(
        self,
        name,
        mujoco_objects=None,
        x_range=(0, 0),
        y_range=(0, 0),
        rotation=None,
        rotation_axis="z",
        ensure_object_boundary_in_range=True,
        ensure_valid_placement=True,
        reference_pos=(0, 0, 0),
        z_offset=0.0,
    ):
        self.x_range = x_range
        self.y_range = y_range
        self.rotation = rotation
        self.rotation_axis = rotation_axis

        super().__init__(
            name=name,
            mujoco_objects=mujoco_objects,
            ensure_object_boundary_in_range=ensure_object_boundary_in_range,
            ensure_valid_placement=ensure_valid_placement,
            reference_pos=reference_pos,
            z_offset=z_offset,
        )

    def _sample_x(self, object_horizontal_radius):
        """
        Samples the x location for a given object

        Args:
            object_horizontal_radius (float): Radius of the object currently being sampled for

        Returns:
            float: sampled x position
        """
        minimum, maximum = self.x_range
        if self.ensure_object_boundary_in_range:
            minimum += object_horizontal_radius
            maximum -= object_horizontal_radius
        return np.random.uniform(high=maximum, low=minimum)

    def _sample_y(self, object_horizontal_radius):
        """
        Samples the y location for a given object

        Args:
            object_horizontal_radius (float): Radius of the object currently being sampled for

        Returns:
            float: sampled y position
        """
        minimum, maximum = self.y_range
        if self.ensure_object_boundary_in_range:
            minimum += object_horizontal_radius
            maximum -= object_horizontal_radius
        return np.random.uniform(high=maximum, low=minimum)

    def _sample_quat(self):
        """
        Samples the orientation for a given object

        Returns:
            np.array: sampled (r,p,y) euler angle orientation

        Raises:
            ValueError: [Invalid rotation axis]
        """
        if self.rotation is None:
            rot_angle = np.random.uniform(high=2 * np.pi, low=0)
        elif isinstance(self.rotation, collections.abc.Iterable):
            rot_angle = np.random.uniform(high=max(self.rotation), low=min(self.rotation))
        else:
            rot_angle = self.rotation

        # Return angle based on axis requested
        if self.rotation_axis == "x":
            return np.array([np.cos(rot_angle / 2), np.sin(rot_angle / 2), 0, 0])
        elif self.rotation_axis == "y":
            return np.array([np.cos(rot_angle / 2), 0, np.sin(rot_angle / 2), 0])
        elif self.rotation_axis == "z":
            return np.array([np.cos(rot_angle / 2), 0, 0, np.sin(rot_angle / 2)])
        else:
            # Invalid axis specified, raise error
            raise ValueError(
                "Invalid rotation axis specified. Must be 'x', 'y', or 'z'. Got: {}".format(self.rotation_axis)
            )

    def sample(self, fixtures=None, reference=None, on_top=True):
        """
        Uniformly sample relative to this sampler's reference_pos or @reference (if specified).

        Args:
            fixtures (dict): dictionary of current object placements in the scene as well as any other relevant
                obstacles that should not be in contact with newly sampled objects. Used to make sure newly
                generated placements are valid. Should be object names mapped to (pos, quat, MujocoObject)

            reference (str or 3-tuple or None): if provided, sample relative placement. Can either be a string, which
                corresponds to an existing object found in @fixtures, or a direct (x,y,z) value. If None, will sample
                relative to this sampler's `'reference_pos'` value.

            on_top (bool): if True, sample placement on top of the reference object. This corresponds to a sampled
                z-offset of the current sampled object's bottom_offset + the reference object's top_offset
                (if specified)

        Return:
            dict: dictionary of all object placements, mapping object_names to (pos, quat, obj), including the
                placements specified in @fixtures. Note quat is in (w,x,y,z) form

        Raises:
            RandomizationError: [Cannot place all objects]
            AssertionError: [Reference object name does not exist, invalid inputs]
        """
        # Standardize inputs
        placed_objects = {} if fixtures is None else copy(fixtures)
        if reference is None:
            base_offset = self.reference_pos
        elif type(reference) is str:
            assert (
                reference in placed_objects
            ), "Invalid reference received. Current options are: {}, requested: {}".format(
                placed_objects.keys(), reference
            )
            ref_pos, _, ref_obj = placed_objects[reference]
            base_offset = np.array(ref_pos)
            if on_top:
                base_offset += np.array((0, 0, ref_obj.top_offset[-1]))
        else:
            base_offset = np.array(reference)
            assert (
                base_offset.shape[0] == 3
            ), "Invalid reference received. Should be (x,y,z) 3-tuple, but got: {}".format(base_offset)

        # Sample pos and quat for all objects assigned to this sampler

        num_objects = len(self.mujoco_objects)
        object_x = torch.rand(num_objects) * (self.x_range[1] - self.x_range[0]) + self.x_range[0]
        object_y = torch.rand(num_objects) * (self.y_range[1] - self.y_range[0]) + self.y_range[0]

        

        for obj in self.mujoco_objects:
            # First make sure the currently sampled object hasn't already been sampled
            assert obj.name not in placed_objects, "Object '{}' has already been sampled!".format(obj.name)

            horizontal_radius = obj.horizontal_radius
            bottom_offset = obj.bottom_offset
            success = False
            
            for i in range(5000):  # 5000 retries
                object_x = self._sample_x(horizontal_radius) + base_offset[0]
                object_y = self._sample_y(horizontal_radius) + base_offset[1]
                object_z = self.z_offset + base_offset[2]
                if on_top:
                    object_z -= bottom_offset[-1]

                # objects cannot overlap
                location_valid = True
                if self.ensure_valid_placement:
                    for (x, y, z), _, other_obj in placed_objects.values():
                        if (
                            np.linalg.norm((object_x - x, object_y - y))
                            <= other_obj.horizontal_radius + horizontal_radius
                        ) and (object_z - z <= other_obj.top_offset[-1] - bottom_offset[-1]):
                            location_valid = False
                            break

                if location_valid:
                    # random rotation
                    quat = self._sample_quat()

                    # multiply this quat by the object's initial rotation if it has the attribute specified
                    if hasattr(obj, "init_quat"):
                        quat = quat_multiply(quat, obj.init_quat)

                    # location is valid, put the object down
                    pos = (object_x, object_y, object_z)
                    placed_objects[obj.name] = (pos, quat, obj)
                    success = True
                    break

            if not success:
                raise RandomizationError("Cannot place all objects ):")

        return placed_objects